In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta  # For RSI calculation
import numpy as np

# List of Thai stock symbols (SET50 for example)
thai_stocks = [ 
    'ADVANC.BK', 'AEONTS.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BANPU.BK', 'BBL.BK', 'BCH.BK', 
    'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK', 'BGRIM.BK', 'BH.BK', 'BJC.BK', 'BLA.BK', 'BPP.BK', 
    'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK', 'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 
    'CRC.BK', 'DOHOME.BK', 'EA.BK', 'EGCO.BK', 'EPG.BK', 'ERW.BK', 'GLOBAL.BK', 'GPSC.BK', 
    'GULF.BK', 'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMT.BK', 'KBANK.BK', 'KCE.BK', 
    'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'M.BK', 'MAJOR.BK', 'MBK.BK', 'MEGA.BK', 'MINT.BK', 'MTC.BK', 
    'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PRM.BK', 'PSH.BK', 'PTG.BK', 'PTT.BK', 'PTTEP.BK', 'PTTGC.BK', 
    'QH.BK', 'RATCH.BK', 'RS.BK', 'SAWAD.BK', 'SC.BK', 'SCC.BK', 'SCGP.BK', 'SCN.BK', 'SINGER.BK', 'SPALI.BK', 
    'SPRC.BK', 'STA.BK', 'STEC.BK', 'STGT.BK', 'SUPER.BK', 'TASCO.BK', 'TCAP.BK', 'THANI.BK', 'TISCO.BK', 
    'TKN.BK', 'TTB.BK', 'TOA.BK', 'TOP.BK', 'TPIPP.BK', 'TRUE.BK', 'TTB.BK', 'TU.BK', 'VGI.BK', 'WHA.BK'
]

# Fetch stock data from yfinance
def get_stock_data(stock, period="5y"):
    return yf.download(stock, period=period)

# Calculate Current Quarterly EPS Growth (C)
def calculate_eps_growth(df):
    df['EPS'] = df['Close'] * 0.05  # Simulating EPS (5% of close price)
    df['EPS_Growth'] = df['EPS'].pct_change(periods=63) * 100  # Approximate quarterly change over 63 trading days
    return df

# Calculate Annual Earnings Growth (A)
def calculate_annual_growth(df):
    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]
    years = len(df) / 252  # Approximate trading days in a year
    df['Annual_Growth'] = ((end_price / start_price) ** (1 / years) - 1) * 100
    return df

# Check if stock hit a new high in the last month (N)
def check_new_high(df):
    last_20_days = df['Close'].tail(20)  # Last 20 trading days (approx. 1 month)
    if df['Close'].iloc[-1] == last_20_days.max():
        return True
    return False

# Check if buy volume exceeds sell volume in the last week (S)
def check_buy_more_than_sell(df):
    df['Buy_Sell_Difference'] = df['Volume'].diff()  # Simulating buy-sell balance using volume difference
    last_5_days_volume = df['Buy_Sell_Difference'].tail(5).sum()  # Last 5 trading days (1 week)
    return last_5_days_volume > 0

# Calculate Relative Strength Rating (L)
def calculate_relative_strength(stocks):
    performances = {}

    for stock in stocks:
        df = get_stock_data(stock, period="1y")  # Get 1 year data for RS Rating
        price_change = ((df['Close'].iloc[-1] - df['Close'].iloc[0]) / df['Close'].iloc[0]) * 100
        performances[stock] = price_change
    
    # Convert to DataFrame for ranking
    performance_df = pd.DataFrame(list(performances.items()), columns=['Stock', '52_Week_Performance'])
    performance_df['RS_Rating'] = performance_df['52_Week_Performance'].rank(pct=True) * 100
    performance_df['RS_Rating'] = performance_df['RS_Rating'].apply(lambda x: min(int(x), 99))

    return performance_df

# Main function to screen stocks based on the given criteria
def screen_stocks(stocks):
    rs_ratings = calculate_relative_strength(stocks)
    screened_stocks = []

    for stock in stocks:
        df = get_stock_data(stock)
        df = calculate_eps_growth(df)
        df = calculate_annual_growth(df)

        # Fetch RS Rating from the pre-calculated data
        rs_rating = rs_ratings.loc[rs_ratings['Stock'] == stock, 'RS_Rating'].values[0]

        # Apply all the screening criteria
        if df['EPS_Growth'].iloc[-1] > 15:  # C: EPS Growth > 15%
            if df['Annual_Growth'].iloc[-1] > 12:  # A: Annual Growth > 12%
                if check_new_high(df):  # N: New high over the last month
                    if check_buy_more_than_sell(df):  # S: Buy volume > Sell volume in the last week
                        if rs_rating > 75:  # RS Rating > 75
                            screened_stocks.append(stock)

    return screened_stocks

# Run the screening process on Thai stocks
screened_stocks = screen_stocks(thai_stocks)

# Display the filtered stocks
print("Stocks that meet the CANSLIM criteria:")
print(screened_stocks)